In [4]:
import boule as bl
import harmonica as hm
import verde as vd
import numpy as np

In [5]:
topography = hm.datasets.fetch_topography_earth().loc[dict(longitude=slice(-180, 180, 10), latitude=slice(-90, 90, 10))]
topography

<xarray.Dataset>
Dimensions:     (latitude: 37, longitude: 73)
Coordinates:
  * longitude   (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0 180.0
  * latitude    (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 80.0 85.0 90.0
Data variables:
    topography  (latitude, longitude) float64 2.762e+03 2.762e+03 ... -4.179e+03

In [24]:
vd.grid_to_table(topography.topography)

AttributeError: 'DataArray' object has no attribute 'data_vars'

In [6]:
ellipsoid = bl.WGS84

In [16]:
spacing = np.mean(np.abs(topography.latitude.values[1:] - topography.latitude.values[:-1]))
spacing

5.0

In [18]:
topo = vd.grid_to_table(topography)

In [19]:
topo["latitude_spherical"], topo["radius"] = ellipsoid.geodetic_to_spherical(topo.longitude, topo.latitude, topo.topography)[1:]

In [20]:
topo

,longitude,latitude,topography,latitude_spherical,radius
0,-180.0,-90.0,2762.0,-90.000000,6.359514e+06
1,-180.0,-85.0,2762.0,-84.966490,6.359678e+06
2,-180.0,-80.0,2762.0,-79.934007,6.360164e+06
3,-180.0,-75.0,2762.0,-74.903549,6.360958e+06
4,-180.0,-70.0,2762.0,-69.876047,6.362034e+06
...,...,...,...,...,...
2696,180.0,70.0,-4179.0,69.875912,6.355093e+06
2697,180.0,75.0,-4179.0,74.903444,6.354017e+06
2698,180.0,80.0,-4179.0,79.933935,6.353223e+06
2699,180.0,85.0,-4179.0,84.966453,6.352737e+06


In [23]:
interp = vd.ScipyGridder(method="nearest").fit((topo.longitude, topo.latitude_spherical), topo.radius)
topo_spherical = interp.grid(spacing=spacing, data_names=["radius"], dims=["latitude", "longitude"])
topo_spherical

<xarray.Dataset>
Dimensions:    (latitude: 37, longitude: 73)
Coordinates:
  * longitude  (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0 180.0
  * latitude   (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 80.0 85.0 90.0
Data variables:
    radius     (latitude, longitude) float64 6.36e+06 6.36e+06 ... 6.353e+06
Attributes:
    metadata:  Generated by ScipyGridder(extra_args=None, method='nearest')

In [68]:
dlats = topography.latitude.values[1:] - topography.latitude.values[:-1]
dlons = topography.longitude.values[1:] - topography.longitude.values[:-1]
dlon = dlons[0]
dlat = dlats[0]
assert np.allclose(dlons, dlon)
assert np.allclose(dlats, dlat)

In [83]:
west = topography.longitude.values - dlon/2
east = topography.longitude.values + dlon/2
west[west > 360] = 360
west[west < -180] = -180
east[east > 360] = 360
east[east < -180] = -180
south = topography.latitude.values - dlat/2
south[south > 90] = 90
south[south < -90] = -90
north = topography.latitude.values + dlat/2
north[north > 90] = 90
north[north < -90] = -90
reference = ellipsoid.geodetic_to_spherical(None, topo.latitude, np.zeros_like(topo.latitude))[2]
radius = ellipsoid.geodetic_to_spherical(None, topo.latitude, topo.topography)[2]
index = np.arange(radius.size)

In [84]:
coordinates = ellipsoid.geodetic_to_spherical(topo.longitude, topo.latitude, np.full_like(topo.latitude, 100e3))

In [85]:
zero_height = np.zeros_like(north)
north_sph = ellipsoid.geodetic_to_spherical(None, north, zero_height)[1]
south_sph = ellipsoid.geodetic_to_spherical(None, south, zero_height)[1]

In [86]:
shape = topography.topography.shape

In [87]:
batch_size = 10
batches = [index[i:i + batch_size] for i in range(0, index.size, batch_size)]
field = np.zeros_like(topo.topography)
for batch in batches:
    tesseroids = []
    for t in batch:
        i, j = np.unravel_index(t, shape)
        tesseroids.append([west[j], east[j], south[i], north[i], reference[t], radius[t]])
    density = np.full_like(batch, 2670)
    field += hm.tesseroid_gravity(coordinates, tesseroids, field="g_z", density=density)
    

ValueError: Invalid tesseroid or tesseroids. The bottom radius boundary can't be greater than the top one.
	Invalid tesseroid: [-1.80000000e+02 -1.77500000e+02 -8.25000000e+01 -7.75000000e+01
  6.35691611e+06  6.35682711e+06]
	Invalid tesseroid: [-1.77500000e+02 -1.72500000e+02 -8.25000000e+01 -7.75000000e+01
  6.35675231e+06  6.35673831e+06]


In [79]:
np.unravel_index(batches[0], shape)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))